In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import os
import pandas as pd
from datetime import datetime
from time import sleep

# Set pandas option to display all columns
pd.set_option('display.max_columns', None)

# Define global DataFrame to collect all data
df = pd.DataFrame()

# Function to run the API call and append data
def api_runner():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'  
    parameters = {
      'start': '1',
      'limit': '5000',
      'convert': 'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '3d98ebe0-3e34-4009-aeb5-e6dc0ff2e368',  # Insert your API Key here 
    }
    
    session = Session()
    session.headers.update(headers)
    
    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        
        # Normalize the JSON response into a pandas DataFrame
        updated_data = pd.json_normalize(data['data']) 
        
        # Use datetime.now() instead of 'now' for timestamp
        updated_data['timestamp'] = pd.to_datetime(datetime.now())
        
        # Declare global df so it doesn't get overwritten
        global df
        
        # Concatenate the new data with the existing df (optimized for empty df)
        if df.empty:
            df = updated_data  # If df is empty, assign updated_data directly
        else:
            df = pd.concat([df, updated_data], ignore_index=True)
            
         # Save the DataFrame to CSV
        if not os.path.isfile(r'/Users/lasya/Automating Crypto Website API.csv'):
            df.to_csv(r'/Users/lasya/Automating Crypto Website API.csv', header='column_names', index=False)
        else:
            df.to_csv(r'/Users/lasya/Automating Crypto Website API.csv', mode='a', header=False, index=False)    
        
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)

# Loop to call the API 333 times
for i in range(333):  # 333 API calls allowed per day
    api_runner()  # Call the API runner function
    print(f'API Runner Completed Successfully - Run {i+1}')  # Print success message for each run
    sleep(60)  # Wait for 60 seconds before the next call

    
# After all runs are complete, you can save the DataFrame to a CSV if desired:
# df.to_csv('crypto_data.csv', index=False)  # Save data to CSV

In [ ]:
df

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x) 

In [ ]:
df

In [ ]:
df3 = df.groupby('name', sort = False)[['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 
                                  'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 
                                  'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d']].mean()
df3

In [ ]:
df4 = df3.stack()

df4

In [ ]:
df4.head(50)

In [ ]:
type(df3)

In [ ]:
type(df4)

In [ ]:
df5 = df4.to_frame(name = 'values')
df5

In [ ]:
type(df5)

In [ ]:
num_rows = len(df5)

In [ ]:
df5.count()

In [ ]:
index = pd.Index(range(num_rows))

df6 = df5.reset_index()
df6

In [ ]:
df7 = df6.rename(columns = {'level_1' : 'percent_change'})
df7                            

In [ ]:
df7['percent_change'] = df7['percent_change'].replace(['quote.USD.percent_change_1h'], ['1h'])
df7['percent_change'] = df7['percent_change'].replace(['quote.USD.percent_change_24h'], ['24h'])
df7['percent_change'] = df7['percent_change'].replace(['quote.USD.percent_change_7d'], ['7d'])
df7['percent_change'] = df7['percent_change'].replace(['quote.USD.percent_change_30d'], ['30d'])
df7['percent_change'] = df7['percent_change'].replace(['quote.USD.percent_change_60d'], ['60d'])
df7['percent_change'] = df7['percent_change'].replace(['quote.USD.percent_change_90d'], ['90d'])


df7

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Filter to include only the top 10 cryptocurrencies by name
top_10 = df7['name'].unique()[:10]  # Get the first 10 unique names (cryptocurrencies)

# Filter the DataFrame to include only these top 10 cryptocurrencies
df_top_10 = df7[df7['name'].isin(top_10)]

# Now plot the filtered data
# Categorical Plot is used to visualize the relationship between categorical variable 
# & one or more numerical variable

sns.catplot(x='percent_change', y='values', hue='name', data= df_top_10, kind='point')

# Show the plot
plt.show()

In [ ]:
df10 = df[['name', 'quote.USD.price', 'timestamp']]
df10 = df10.query("name == 'Bitcoin'")
df10

In [ ]:
sns.set_theme(style = "darkgrid")
sns.lineplot(x = 'timestamp', y = 'quote.USD.price', data = df10)